In [194]:
import pandas as pd

In [196]:
df = pd.read_csv("INVENTARIOTHIAGO - Página16.csv")
df.head(2)

,DATA VENDA,ITEM,QUANTIDADE,CATEGORIA,STATUS,VALOR,PAGO
0,26/11/24,Xbox One Gears,1,Console,Enviado,"R$ 3.350,00","R$ 3.350,00"
1,26/11/24,PS4 Pro Spider Man,1,Console,Reservado,"R$ 3.750,00",NaN


In [198]:
df.columns

Index(['DATA VENDA', 'ITEM', 'QUANTIDADE', 'CATEGORIA', 'STATUS', 'VALOR',
       'PAGO'],
      dtype='object')

In [200]:
# Cleaning
def clean_currency(column):
    return column.str.replace('R$', '', regex=False).str.replace(',00', '', regex=False).str.replace('.', '', regex=False).str.strip()

# Aplicando a função nas colunas 'VALOR' e 'PAGO'
df['VALOR'] = clean_currency(df['VALOR'])
df['PAGO'] = clean_currency(df['PAGO'])
df["PAGO"] =  df["PAGO"].fillna(0)
df['VALOR'] = df['VALOR'].astype(int)
df['PAGO'] = df['PAGO'].astype(int)
df['DATA VENDA'] = pd.to_datetime(df['DATA VENDA'], format="%d/%m/%y")

In [202]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   DATA VENDA  15 non-null     datetime64[ns]
 1   ITEM        15 non-null     object        
 2   QUANTIDADE  15 non-null     int64         
 3   CATEGORIA   15 non-null     object        
 4   STATUS      15 non-null     object        
 5   VALOR       15 non-null     int64         
 6   PAGO        15 non-null     int64         
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 972.0+ bytes


In [204]:
qtd = df['QUANTIDADE'].sum()
print(f"Soma total dos itens vendidos {qtd}")

Soma total dos itens vendidos 16


In [206]:
faturamento = df['VALOR'].sum()
recebido = df['PAGO'].sum()
restante = faturamento - recebido

print(f"total de faturamento R$:{faturamento},00")
print(f"total de recebido R$:{recebido},00")
print(f"total de restante R$:{restante},00")


total de faturamento R$:64145,00
total de recebido R$:30295,00
total de restante R$:33850,00


In [210]:
resumo_df = pd.DataFrame({
    "Descrição": ["Faturamento Total", "Total Recebido", "Total Restante"],
    "Valor": [faturamento, recebido, restante]
})
resumo_df['Valor'] = resumo_df['Valor'].apply(lambda x: f"R$ {x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))
resumo_df

,Descrição,Valor
0,Faturamento Total,"R$ 64.145,00"
1,Total Recebido,"R$ 30.295,00"
2,Total Restante,"R$ 33.850,00"


In [218]:
bycategory = df.groupby("CATEGORIA")[["QUANTIDADE","VALOR"]].sum()
bycategory['VALOR'] = bycategory['VALOR'].apply(lambda x: f"R$ {x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))
bycategory = bycategory.reset_index()
bycategory

,CATEGORIA,QUANTIDADE,VALOR
0,Console,8,"R$ 43.950,00"
1,Controle,4,"R$ 11.895,00"
2,Jogo Edição,4,"R$ 8.300,00"


In [222]:
bydate = df.groupby("DATA VENDA")[["QUANTIDADE","VALOR"]].sum()
bydate['VALOR'] = bydate['VALOR'].apply(lambda x: f"R$ {x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))
bydate = bydate.reset_index()
bydate

,DATA VENDA,QUANTIDADE,VALOR
0,2024-11-26,10,"R$ 52.550,00"
1,2024-11-27,2,"R$ 1.650,00"
2,2024-11-29,1,"R$ 2.500,00"
3,2024-12-03,3,"R$ 7.445,00"


In [224]:
bystats = df.groupby("STATUS")[["QUANTIDADE", "VALOR"]].sum()
bystats['VALOR'] = bystats['VALOR'].apply(lambda x: f"R$ {x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))
bystats = bystats.reset_index()
bystats

,STATUS,QUANTIDADE,VALOR
0,Enviado,5,"R$ 24.250,00"
1,Pago,2,"R$ 4.545,00"
2,Parcialmente Pago,1,"R$ 2.500,00"
3,Reservado,8,"R$ 32.850,00"


In [226]:
# Exportar múltiplos dataframes para um único arquivo Excel
with pd.ExcelWriter("reportmethodoutput.xlsx", engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='listadevenda', index=False)
    resumo_df.to_excel(writer, sheet_name='Faturamento', index=False)
    bycategory.to_excel(writer, sheet_name='Categorias', index=False)
    bystats.to_excel(writer, sheet_name='status', index=False)
    bydate.to_excel(writer, sheet_name='data', index=False)



In [257]:
df = pd.read_csv("resumoreportthiago.xlsx - Página1.csv")

In [259]:
df

,CATEGORIA,NOVOS,ABERTOS,TOTAL
0,Consoles,6,16,22
1,Controles,12,5,17
2,Jogos e Edições,32,60,92
3,Outros,0,2,2


In [261]:
totals = df[["NOVOS", "ABERTOS", "TOTAL"]].sum()

# Calculando percentuais por categoria
df["PERC_NOVOS"] = (df["NOVOS"] / totals["NOVOS"]) * 100
df["PERC_ABERTOS"] = (df["ABERTOS"] / totals["ABERTOS"]) * 100
df["PERC_TOTAL"] = (df["TOTAL"] / totals["TOTAL"]) * 100

# Exibindo o DataFrame atualizado
print("Tabela com Percentuais:")
print(df)

# Exibindo totais gerais
print("\nTotais Gerais:")
print(totals)

Tabela com Percentuais:
         CATEGORIA  NOVOS  ABERTOS  TOTAL  PERC_NOVOS  PERC_ABERTOS  \
0         Consoles      6       16     22        12.0     19.277108   
1        Controles     12        5     17        24.0      6.024096   
2  Jogos e Edições     32       60     92        64.0     72.289157   
3           Outros      0        2      2         0.0      2.409639   

   PERC_TOTAL  
0   16.541353  
1   12.781955  
2   69.172932  
3    1.503759  

Totais Gerais:
NOVOS       50
ABERTOS     83
TOTAL      133
dtype: int64


In [245]:
grouped = df.groupby("CATEGORIA").sum()
totals = grouped.sum()
totals

Novos       50
Abertos     83
Total      133
dtype: int64

In [255]:
grouped["PERC_QUANTIDADE"] = (grouped["Novos"] / totals["Total"]) * 100
grouped.round(3)

,Novos,Abertos,Total,PERC_QUANTIDADE
CATEGORIA,,,,
Consoles,6,16,22,4.511
Controles,12,5,17,9.023
Jogos e Edições,32,60,92,24.060
Outros,0,2,2,0.000
